In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords

from tqdm import tqdm
tqdm.pandas()

In [2]:
# df = pd.read_csv("amazon_reviews-clean_1.csv")
df = pd.read_csv("Amazon_review.csv")
# df = df.head(1000)
df.head(2)

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,ReviewSummary,ReviewText
0,B0000GHNUE,A3D7GP8AS2PRIT,S.C.,1,1,5,2010-03-15 00:00:00,Hot but still flavorful,We had the Green Chile Habanero one at a local...
1,B000TTDDWE,AA1TQ4QJ4Y94P,barb,0,0,5,2010-03-04 00:00:00,Yummy,I bought this item at Costco at Christmas time...


In [3]:
df.shape

(568454, 9)

In [4]:
df.isna().sum()

ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
ReviewSummary             27
ReviewText                 0
dtype: int64

In [5]:
df = df[~df["ReviewSummary"].isna()]
df.head(2)
df.shape

(568427, 9)

In [6]:
df = df[~df.duplicated()]
df.shape

(568146, 9)

In [7]:
df.isna().sum()

ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
ReviewSummary              0
ReviewText                 0
dtype: int64

In [8]:
X = df[["ReviewSummary", "ReviewText"]]
y = df[["Score"]]

In [9]:
X["Final_Summary"] = X["ReviewSummary"] + " " + X["ReviewText"]
X.drop(["ReviewSummary", "ReviewText"], axis=1, inplace = True)
X.head(2)

/var/folders/9h/c3j30yj94rsgzs88yqlpgv5m0000gn/T/ipykernel_51192/3793120357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Final_Summary"] = X["ReviewSummary"] + " " + X["ReviewText"]
/var/folders/9h/c3j30yj94rsgzs88yqlpgv5m0000gn/T/ipykernel_51192/3793120357.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(["ReviewSummary", "ReviewText"], axis=1, inplace = True)


,Final_Summary
0,Hot but still flavorful We had the Green Chile...
1,Yummy I bought this item at Costco at Christma...


In [10]:
X.shape

(568146, 1)

In [11]:
X.isna().sum()

Final_Summary    0
dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=100)

In [13]:
X_train.head()

,Final_Summary
517345,Ive' bought hundreds We are big Pepperoni fans...
176230,a hit with my Grandson Can't say that I've tri...
543075,Excellent Low Carb Bread This bread is excelle...
274014,babybmom My 9 mo. old daughter only eats earth...
29162,Gumbo Mix This gumbo is fantastic. Everyone t...


In [14]:
X_test.head()

,Final_Summary
25886,Can't ask for more Great product and price. I...
472292,Nothing Special Ordered these thinking they we...
388572,Decaf? This can't be decaf! This is one of the...
152448,Cookie Crumbs Walkers cookies or biscuits alwa...
80489,MMM - Not your typical store bought sauce - Go...


In [15]:
# X_train["Final_Summary"] = X_train["Final_Summary"].progress_apply(lambda x: re.sub("[^a-zA-Z]", " ", str(x)))
# X_train.head(2)

In [16]:
# X_test["Final_Summary"] = X_test["ReviewSummary"] + " " +X_test["ReviewText"]
# X_test.drop(['ReviewSummary', 'ReviewText'], axis=1, inplace=True)
# X_test.head()

In [17]:
def preprocess(X_train, y_train):
    import nltk
    from nltk.stem import WordNetLemmatizer, PorterStemmer
    from nltk.corpus import stopwords
    lemmatize = WordNetLemmatizer()
    stemmer = PorterStemmer()
    # Implement RegEx to remove all special chars
    X_train["Final_Summary"] = X_train["Final_Summary"].apply(lambda x: re.sub("[^a-zA-Z]", " " , str(x)).lower())
    # Implementing the lemmatization, stopwords on the review text column
    from tqdm import tqdm
    tqdm.pandas()
#     X_train["Final_Summary_Reg_Lower_Lem_StopWords"] = X_train["Final_Summary_Reg_Lower"].progress_apply(lambda row: list(set([" ".join(lemmatize.lemmatize(x)) for x in row.split(" ") if(x not in stopwords.words("english" or x != "") )])))
#     X_train["Final_Summary_Reg_Lower_Lem_StopWords_NullRemoval"] = X_train["Final_Summary_Reg_Lower_Lem_StopWords"].apply(lambda row: " ".join(x) for x in row if x !="") 
    X_train["Final_Summary_Reg_Lower_Lem_StopWords"] = X_train["Final_Summary"].progress_apply(lambda row: " ".join(lemmatize.lemmatize(x) for x in row.split(" ") if(x not in stopwords.words("english" or x != ""))))
    return X_train                                                                                                     
#     return pd.concat([X_train, y_train], axis=1)
                                                                                                                                                                                                           

In [18]:
clean_data_test = preprocess(X_test, y_test)
cleaned_data_test = clean_data_test.iloc[:,-2:]
cleaned_data_test  = pd.concat([clean_data_test, y_test], axis =1)
cleaned_data_test.to_csv("cleaned_data_test.csv")

100%|███████████████████████████████████| 113630/113630 [50:25<00:00, 37.56it/s]


In [19]:
clean_data_train = preprocess(X_train, y_train)
cleaned_data_train = clean_data_train.iloc[:,-2:]
cleaned_data_train = pd.concat([cleaned_data_train, y_train], axis=1)
cleaned_data_train.to_csv("cleaned_data_train.csv")

100%|█████████████████████████████████| 454516/454516 [3:51:49<00:00, 32.68it/s]
